<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/BiDAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk import wordnet
from nltk.tokenize import word_tokenize
import string

import torch
import numpy as np
import pandas as pd
import pickle
import re, os, string, typing, gc, json
import spacy
from collections import Counter



stemmer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Build a list of dictionaries with ( context , question , label ) as keys**

In [21]:
def parse_data(dataframe):
  QA_list=[]

  for i in range(0,len(dataframe['index'])):
    QA_dict={}
    QA_dict['id']=i
    QA_dict['context']=dataframe['context_translated'][i]
    QA_dict['question']=dataframe['question_translated'][i]
    QA_dict['label']=[dataframe['answer_start'][0],dataframe['answer_end'][i]]
    QA_list.append(QA_dict)
  return QA_list

# **Function to pre-process paragraph of text**

In [22]:

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', '', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # define punctuation
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


        preprocessed_text= ""
        for char in document:
          if char not in punctuations:
              preprocessed_text = preprocessed_text+ char
        
        return preprocessed_text

# **Function to create list of Context, Question and Context+Question lists**

In [23]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['text_translated']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  lis=context+question
  return lis,context,question,answer

# **Function to build vocabulary of words , word to index and index to word mapping**

In [24]:
def build_word_vocab(lis):
   sentence_vocab=[]
   for sentences in lis:
      tokenized=sentences.split()
      sentence_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   word_vocab=[]
   for line in lis: 
     words=line.split()
     for word in words:
        if word not in word2idx: 
          word2idx[word] = len(word2idx)
        if word not in word_vocab:
          word_vocab.append(word)


   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab


# **Function to create character vocabulary and character to index mapping**

In [25]:
def build_char_vocab(vocab_text):

    chars = []
    for sent in vocab_text:
        for ch in sent:
            chars.append(ch)

    char_counter = Counter(chars)
    char_vocab = sorted(char_counter, key=char_counter.get, reverse=True)
    print(f"raw-char-vocab: {len(char_vocab)}")
    high_freq_char = [char for char, count in char_counter.items() if count>=20]
    char_vocab = list(set(char_vocab).intersection(set(high_freq_char)))
    print(f"char-vocab-intersect: {len(char_vocab)}")
    char_vocab.insert(0,'<unk>')
    char_vocab.insert(1,'<pad>')
    char2idx = {char:idx for idx, char in enumerate(char_vocab)}
    print(f"char2idx-length: {len(char2idx)}")
    
    return char2idx, char_vocab

# **Function to create id's for pre-processed context words**

In [26]:
def context_to_ids(text, word2idx):
   
    context_tokens=text.split()
    # for paragraph in stripped:
    #     tokenized=word_tokenize(paragraph)
    #     context_tokens.append(tokenized)
    
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

# **Function to create id's for pre-processed questions**

In [27]:
def question_to_ids(text, word2idx):
 

    question_tokens=text.split()
    question_ids = [word2idx[word] for word in question_tokens]
    
    assert len(question_ids) == len(question_tokens)
    return question_ids

# **Function to create id's for answers**

In [28]:
def answer_to_ids(text, word2idx):
 

    answer_tokens=text.split()
    answer_ids = [word2idx[word] for word in answer_tokens]
    
    assert len(answer_ids) == len(answer_tokens)
    return answer_ids

In [9]:
from google.colab import files
uploaded = files.upload()

Saving partially_cleaned.csv to partially_cleaned.csv


In [10]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [11]:
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)
data

,context_translated,question_translated,text_translated
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද?,උතුරු අත්ලාන්තික්
1,"2002 දී ප්‍රදර්ශනය ආරම්භ වූ දා සිට, අයිඩල් ජයග...",ඇමරිකානු අයිඩල් තරඟකරු ක්‍රිස් ඩොට්‍රි කුමන ප්...,උතුරු කැරොලිනාව
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්‍රිකයන...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද?,දේශීය කලාකරුවන්
4,ටුවාලුආන් භාෂාව සහ ඉංග්‍රීසි යනු ටුවාලු හි ජාත...,ඉංග්‍රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුම...,ටුවාලුවාන්
...,...,...,...
19995,"දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට, සාමාජික...",ලිබරල් ප්‍රජාතන්ත්‍රවාදයට රාජ්‍යයක් දක්වන සහයෝ...,"පිහිටුවීමට, සාමාජිකත්වය ප්‍රකාශ කිරීමට හෝ උද් ..."
19996,පැරණි ඉංග්‍රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නි...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්‍රීසි ලි...,රූනික් පද්ධතිය
19997,රූනික් පද්ධතිය... ඇරිස්ටෝෆනේස්ගේ නාට්‍ය ග්‍රීක...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්‍...,ගිම්හානය


In [12]:
df=pd.read_csv("partially_cleaned.csv")
df=df.drop(['Unnamed: 0'],axis=1)
df

,index,context_translated,question_translated,text_translated,answer_start,answer_end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,56,66
2,8,ජේන් ඔස්ටන් ප්රධාන තදාසන්න ප්රදේශ අතර නිරිත දෙ...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්රදේශය කුමක්ද,මිඩ්ලෝතියන්,51,62
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්ර පාක්ෂිකයන...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්රාන්ත...,හත,326,328
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්රංශයේ අවනීතිය ස...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,157,171
...,...,...,...,...,...,...
9314,19990,1900 වන විට මිනිසුන් 7531 ක් නගරයේ වාසය කළහ 19...,2006 දී ටක්සන්ගේ ජනගහනය කොපමණද,535000 කි,511,520
9315,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,313,318
9316,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,131,149
9317,19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...,27,101


In [13]:
df=df.rename(columns={"index": "id", "context_translated": "context","question_translated":"question","text_translated":"answer","answer_start":"start","answer_end":"end"})
df

,id,context,question,answer,start,end
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,56,66
2,8,ජේන් ඔස්ටන් ප්රධාන තදාසන්න ප්රදේශ අතර නිරිත දෙ...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්රදේශය කුමක්ද,මිඩ්ලෝතියන්,51,62
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්ර පාක්ෂිකයන...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්රාන්ත...,හත,326,328
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්රංශයේ අවනීතිය ස...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,157,171
...,...,...,...,...,...,...
9314,19990,1900 වන විට මිනිසුන් 7531 ක් නගරයේ වාසය කළහ 19...,2006 දී ටක්සන්ගේ ජනගහනය කොපමණද,535000 කි,511,520
9315,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,313,318
9316,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,131,149
9317,19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...,27,101


In [14]:
label=[]
for i in range(0,len(df['id'])):
  lis=[df['start'][i],df['end'][i]]
  label.append(lis) 

df['label_idx']=label
df

,id,context,question,answer,start,end,label_idx
0,0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17,"[0, 17]"
1,4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,56,66,"[56, 66]"
2,8,ජේන් ඔස්ටන් ප්රධාන තදාසන්න ප්රදේශ අතර නිරිත දෙ...,රිච්මන්ඩ් හි නිරිතදිග තදාසන්න ප්රදේශය කුමක්ද,මිඩ්ලෝතියන්,51,62,"[51, 62]"
3,9,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනිය මිත්ර පාක්ෂිකයන...,දෙවන ලෝක යුද්ධයෙන් පසු ජර්මනියේ මුලින් ප්රාන්ත...,හත,326,328,"[326, 328]"
4,21,නැපෝලියන් පශ්චාත් විප්ලවවාදී ප්රංශයේ අවනීතිය ස...,දහස් ගණනකට යුද්ධය හා මරණය පිළිබඳ අපේක්ෂාවෙන් න...,ඔහුගේ විවේචකයෝ,157,171,"[157, 171]"
...,...,...,...,...,...,...,...
9314,19990,1900 වන විට මිනිසුන් 7531 ක් නගරයේ වාසය කළහ 19...,2006 දී ටක්සන්ගේ ජනගහනය කොපමණද,535000 කි,511,520,"[511, 520]"
9315,19991,තෙවන පරම්පරාව ෆයර්වෙයාර් හෝ යූඑස්බී සම්බන්ධතාව...,ලිපිගොනු මාරු කිරීමේදී ෆයර්වෙයාර් භාවිතය මුලින...,පස්වන,313,318,"[313, 318]"
9316,19992,ඔරලෝසු මාරුව හෘදයාබාධ ඇතිවීමේ අවදානම සියයට 10 ...,සෘතුමය වෙනස්කම් මගින් කඩාකප්පල් වන ස්වාභාවික ර...,සර්කැඩියානු රිද්මය,131,149,"[131, 149]"
9317,19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...,27,101,"[27, 101]"


In [15]:
msk = np.random.rand(len(df)) < 0.90661835
train_df = df[msk]
train_df=train_df.reset_index()
valid_df = df[~msk]
valid_df=valid_df.reset_index()

In [16]:
print(len(train_df))
print(len(valid_df))


8390
929


# **Data Preprocessing**

In [17]:
from torch import nn
import torch
import numpy as np
import pandas as pd
import pickle, time
import re, os, string, typing, gc, json
import torch.nn.functional as F
import spacy
from sklearn.model_selection import train_test_split
from collections import Counter
nlp = spacy.load('en')
%load_ext autoreload
%autoreload 2

**Gather text for create word vocabulary using the full dataset**

In [18]:
lis,c,q,a = gather_text_for_vocab(data)

vocab_text=lis
print("Number of sentences in dataset: ", len(vocab_text))

Number of sentences in dataset:  40000


**Word-index mapping , index-word mapping , word vocabulary**

In [29]:
word2idx, idx2word, word_vocab = build_word_vocab(vocab_text)

**Character-index mapping , character vocabulary**

In [30]:
char2idx, char_vocab = build_char_vocab(vocab_text)

raw-char-vocab: 794
char-vocab-intersect: 191
char2idx-length: 193


**Creating context and question id's for training dataset**

In [31]:
q=0
context_ids_train=[]
for i in range(0,len(train_df['context'])):
    try:                                                        
        ids = context_to_ids(train_df['context'][i] , word2idx)
        context_ids_train.append(ids)
        
    except Exception as e:
        context_ids_train.append(e)
        q=q+1
        continue

In [32]:
r=0
question_ids_train=[]
for i in range(0,len(train_df['question'])):
    try:                                                        
        ids = question_to_ids(train_df['question'][i] , word2idx)
        question_ids_train.append(ids)
        
    except Exception as e:
        question_ids_train.append(e)
        r=r+1
        continue

In [33]:
w=0
answer_ids_train=[]
for i in range(0,len(train_df['question'])):
    try:                                                        
        ids = answer_to_ids(train_df['answer'][i] , word2idx)
        answer_ids_train.append(ids)
        
    except Exception as e:
        answer_ids_train.append(e)
        w=w+1
        continue

**Creating context and question id's for validation dataset**

In [34]:
p=0
context_ids_valid=[]
for i in range(0,len(valid_df['context'])):
    try:                                                        
        ids = context_to_ids(valid_df['context'][i] , word2idx)
        context_ids_valid.append(ids)
        
    except Exception as e:
        context_ids_valid.append(e)
        p=p+1
        continue

In [35]:
s=0
question_ids_valid=[]
for i in range(0,len(valid_df['question'])):
    try:                                                        
        ids = question_to_ids(valid_df['question'][i] , word2idx)
        question_ids_valid.append(ids)
        
    except Exception as e:
        question_ids_valid.append(e)
        s=s+1
        continue

In [36]:
v=0
answer_ids_valid=[]
for i in range(0,len(valid_df['question'])):
    try:                                                        
        ids = answer_to_ids(valid_df['answer'][i] , word2idx)
        answer_ids_valid.append(ids)
        
    except Exception as e:
        answer_ids_valid.append(e)
        v=v+1
        continue

**Adding context,question id rows to train and valid datasets**

In [37]:
train_df['context_ids'] = context_ids_train
train_df['question_ids'] = question_ids_train


In [38]:
valid_df['context_ids'] = context_ids_valid
valid_df['question_ids'] = question_ids_valid


In [39]:
train_df.to_pickle('bidaftrain.pkl')
valid_df.to_pickle('bidafvalid.pkl')

with open('bidafw2id.pickle','wb') as handle:
    pickle.dump(word2idx, handle)

with open('bidafc2id.pickle','wb') as handle:
    pickle.dump(char2idx, handle)


In [40]:
train_df = pd.read_pickle('bidaftrain.pkl')
valid_df = pd.read_pickle('bidafvalid.pkl')

with open('bidafw2id.pickle','rb') as handle:
    word2idx = pickle.load(handle)
with open('bidafc2id.pickle','rb') as handle:
    char2idx = pickle.load(handle)

idx2word = {v:k for k,v in word2idx.items()}

In [45]:
class SquadDataset:
    '''
    - Creates batches dynamically by padding to the length of largest example
      in a given batch.
    - Calulates character vectors for contexts and question.
    - Returns tensors for training.
    '''
    
    def __init__(self, data, batch_size):
        
        self.batch_size = batch_size
        data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
        self.data = data
        
        
    def __len__(self):
        return len(self.data)
    
    def make_char_vector(self, max_sent_len, max_word_len, sentence):
        
        char_vec = torch.ones(max_sent_len, max_word_len).type(torch.LongTensor)
        
        for i, word in enumerate(sentence.split()):
            for j, ch in enumerate(word):
                char_vec[i][j] = char2idx.get(ch, 0)
        
        return char_vec    
    
    def get_span(self, text):
        
        text = nlp(text, disable=['parser','tagger','ner'])
        span = [(w.idx, w.idx+len(w.text)) for w in text]

        return span

    def __iter__(self):
        '''
        Creates batches of data and yields them.
        
        Each yield comprises of:
        :padded_context: padded tensor of contexts for each batch 
        :padded_question: padded tensor of questions for each batch 
        :char_ctx & ques_ctx: character-level ids for context and question
        :label: start and end index wrt context_ids
        :context_text,answer_text: used while validation to calculate metrics
        :ids: question_ids for evaluation
        
        '''
        
        for batch in self.data:
            
            spans = []
            ctx_text = []
            answer_text = []
            
            for ctx in batch.context:
                ctx_text.append(ctx)
                spans.append(self.get_span(ctx))
            
            for ans in batch.answer:
                answer_text.append(ans)
                
            
            max_context_len = max([len(ctx) for ctx in batch.context_ids])
            padded_context = torch.LongTensor(len(batch), max_context_len).fill_(1)
            
            for i, ctx in enumerate(batch.context_ids):
                padded_context[i, :len(ctx)] = torch.LongTensor(ctx)

            max_word_ctx = 0
            for context in batch.context:
                for word in nlp(context, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ctx:
                        max_word_ctx = len(word.text)
            
            char_ctx = torch.ones(len(batch), max_context_len, max_word_ctx).type(torch.LongTensor)
            for i, context in enumerate(batch.context):
                char_ctx[i] = self.make_char_vector(max_context_len, max_word_ctx, context)
            
            max_question_len = max([len(ques) for ques in batch.question_ids])
            padded_question = torch.LongTensor(len(batch), max_question_len).fill_(1)
            
            for i, ques in enumerate(batch.question_ids):
                padded_question[i, :len(ques)] = torch.LongTensor(ques)
                
            max_word_ques = 0
            for question in batch.question:
                for word in nlp(question, disable=['parser','tagger','ner']):
                    if len(word.text) > max_word_ques:
                        max_word_ques = len(word.text)
            
            char_ques = torch.ones(len(batch), max_question_len, max_word_ques).type(torch.LongTensor)
            for i, question in enumerate(batch.question):
                char_ques[i] = self.make_char_vector(max_question_len, max_word_ques, question)
            
            ids = list(batch.id)  
            label = torch.LongTensor(list(batch.label_idx))
            
            yield (padded_context, padded_question, char_ctx, char_ques, label, ctx_text, answer_text, ids)

In [46]:
train_dataset = SquadDataset(train_df, 16)

In [47]:
valid_dataset = SquadDataset(valid_df, 16)

In [48]:
a = next(iter(train_dataset))
a

(tensor([[   2,    3,    4,  ...,    1,    1,    1],
         [ 255,  256,   15,  ...,    1,    1,    1],
         [ 495,  496,  497,  ...,    1,    1,    1],
         ...,
         [ 287, 1130, 1656,  ...,    1,    1,    1],
         [1694, 1695,   98,  ...,    1,    1,    1],
         [1766,   98, 1767,  ...,    1,    1,    1]]),
 tensor([[ 5730,   592,  1194,  2625,  4835, 82141,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1],
         [  257,   380,    55,   259,   233,   260,   256,  9417,     1,     1,
              1,     1,     1,     1,     1,     1],
         [  495,   496,   497,   498,   588,  2623,   563, 30580, 82143,     1,
              1,     1,     1,     1,     1,     1],
         [  813,   814,  1648,    50,  3111,   341, 18194, 11617, 28808,   821,
            354,   812,   198, 15046,     1,     1],
         [  898, 75721,  4546,   304,    70,   908, 23829, 82147,     1,     1,
              1,     1,     1,     1,     1,     1],
 

# **Start of layers**

In [ ]:
# !pip install PyDrive

In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# downloaded = drive.CreateFile({'id':"1Rr1mgeyu7V7xv5Z5CgYsPVzaTBfWZ08s"})  
# downloaded.GetContentFile('cc.si.300.bin')    

In [49]:
pip install fasttext

     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3040939 sha256=dc21989e569f422f7eb6f6285e72368af676d47ab6e60c7652281c819c52e3cc
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [50]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [51]:
import fasttext
dir="drive/My Drive/Word_Embeddings/"
model_fastText=fasttext.load_model("drive/My Drive/Word_Embeddings/Copy of cc.si.300.bin", )

In [ ]:
# with open("cc.si.300.vec", "r") as f:
#     array = []
#     for line in f:
#         array.append(line)

In [ ]:

# def get_fastText_dict():
#     '''
#     Parses the glove word vectors text file and returns a dictionary with the words as
#     keys and their respective pretrained word vectors as values.

#     '''
#     fastText_dict = {}
#     with open("cc.si.300.vec", "r", encoding="utf-8") as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             vector = np.asarray(values[1:], "float32")
#             fastText_dict[word] = vector
            
#     f.close()
    
#     return fastText_dict

In [ ]:
# fastText_dict = get_fastText_dict()

In [ ]:
# type(fastText_dict)

dict

In [ ]:

# def create_weights_matrix(fastText_dict):
#     '''
#     Creates a weight matrix of the words that are common in the fastText vocab and
#     the dataset's vocab. Initializes OOV words with a zero vector.
#     '''
#     weights=[]
#     #weights_matrix = np.zeros((len(word_vocab), 300))
#     words_found = 0
#     for word in word_vocab:
#         # try:
#           w=fastText_dict[word]
#           weights.append(w)

#         #weights_matrix[i] = fastText_dict[word]
#           words_found += 1
#         # except:
#         #    pass
        
#     # return weights_matrix, words_found
#     return weights,words_found

In [52]:
def create_weights_matrix():
    '''
    Creates a weight matrix of the words that are common in the GloVe vocab and
    the dataset's vocab. Initializes OOV words with a zero vector.
    '''
    weights_matrix = np.zeros((len(word_vocab), 300))
    words_found = 0
    for i, word in enumerate(word_vocab):
        try:
            weights_matrix[i] = model_fastText[word]
            words_found += 1
        except:
            pass
        
    return weights_matrix, words_found

In [53]:

weights_matrix, words_found = create_weights_matrix()
print("Words found in the GloVe vocab: " ,words_found)

Words found in the GloVe vocab:  85812


In [54]:
np.save('bidafglove.npy', weights_matrix)

In [ ]:
len(word_vocab)

85812

In [55]:
from torch import nn
import torch

In [56]:
class CharacterEmbeddingLayer(nn.Module):
    
    def __init__(self, char_vocab_dim, char_emb_dim, num_output_channels, kernel_size):
        
        super().__init__()
        
        self.char_emb_dim = char_emb_dim
        
        self.char_embedding = nn.Embedding(char_vocab_dim, char_emb_dim, padding_idx=1)
        
        self.char_convolution = nn.Conv2d(in_channels=1, out_channels=300, kernel_size=kernel_size)
        
        self.relu = nn.ReLU()
    
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        # x = [bs, seq_len, word_len]
        # returns : [batch_size, seq_len, num_output_channels]
        # the output can be thought of as another feature embedding of dim 100.
        
        batch_size = x.shape[0]
        
        x = self.dropout(self.char_embedding(x))
        # x = [bs, seq_len, word_len, char_emb_dim]
        
        # following three operations manipulate x in such a way that
        # it closely resembles an image. this format is important before 
        # we perform convolution on the character embeddings.
        
        x = x.permute(0,1,3,2)
        # x = [bs, seq_len, char_emb_dim, word_len]
        
        x = x.view(-1, self.char_emb_dim, x.shape[3])
        # x = [bs*seq_len, char_emb_dim, word_len]
        
        x = x.unsqueeze(1)
        # x = [bs*seq_len, 1, char_emb_dim, word_len]
        
        # x is now in a format that can be accepted by a conv layer. 
        # think of the tensor above in terms of an image of dimension
        # (N, C_in, H_in, W_in).
        
        x = self.relu(self.char_convolution(x))
        # x = [bs*seq_len, out_channels, H_out, W_out]
        
        x = x.squeeze()
        # x = [bs*seq_len, out_channels, W_out]
                
        x = F.max_pool1d(x, x.shape[2]).squeeze()
        # x = [bs*seq_len, out_channels, 1] => [bs*seq_len, out_channels]
                
        x = x.view(batch_size, -1, x.shape[-1])
        # x = [bs, seq_len, out_channels]
        # x = [bs, seq_len, features] = [bs, seq_len, 100]
        
        
        return x

In [57]:
class HighwayNetwork(nn.Module):
    
    def __init__(self, input_dim, num_layers=2):
        
        super().__init__()
        
        self.num_layers = num_layers
        
        self.flow_layer = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.gate_layer = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        
    def forward(self, x):
        
        for i in range(self.num_layers):
            
            flow_value = F.relu(self.flow_layer[i](x))
            gate_value = torch.sigmoid(self.gate_layer[i](x))
            
            x = gate_value * flow_value + (1-gate_value) * x
        
        return x

In [58]:
class ContextualEmbeddingLayer(nn.Module):
    
    def __init__(self, input_dim, hidden_dim):
        
        super().__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        
        self.highway_net = HighwayNetwork(input_dim)
        
    def forward(self, x):
        # x = [bs, seq_len, input_dim] = [bs, seq_len, emb_dim*2]
        # the input is the concatenation of word and characeter embeddings
        # for the sequence.
        
        highway_out = self.highway_net(x)
        # highway_out = [bs, seq_len, input_dim]
        
        outputs, _ = self.lstm(highway_out)
        # outputs = [bs, seq_len, emb_dim*2]
        
        return outputs

In [59]:
class BiDAF(nn.Module):
    
    def __init__(self, char_vocab_dim, emb_dim, char_emb_dim, num_output_channels, 
                 kernel_size, ctx_hidden_dim, device):
        '''
        char_vocab_dim = len(char2idx)
        emb_dim = 100
        char_emb_dim = 8
        num_output_chanels = 100
        kernel_size = (8,5)
        ctx_hidden_dim = 100
        '''
        super().__init__()
        
        self.device = device
        
        self.word_embedding = self.get_glove_embedding()
        
        self.character_embedding = CharacterEmbeddingLayer(char_vocab_dim, char_emb_dim, 
                                                      num_output_channels, kernel_size)
        
        self.contextual_embedding = ContextualEmbeddingLayer(emb_dim*2, ctx_hidden_dim)
        
        self.dropout = nn.Dropout()
        
        self.similarity_weight = nn.Linear(emb_dim*6, 1, bias=False)
        
        self.modeling_lstm = nn.LSTM(emb_dim*8, emb_dim, bidirectional=True, num_layers=2, batch_first=True, dropout=0.2)        
        self.output_start = nn.Linear(emb_dim*10, 1, bias=False)
        
        self.output_end = nn.Linear(emb_dim*10, 1, bias=False)
        
        self.end_lstm = nn.LSTM(emb_dim*2, emb_dim, bidirectional=True, batch_first=True)
        
    
    def get_glove_embedding(self):
        
        weights_matrix = np.load('bidafglove.npy')
        num_embeddings, embedding_dim = weights_matrix.shape
        embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights_matrix).to(self.device),freeze=True)

        return embedding
        
    def forward(self, ctx, ques, char_ctx, char_ques):
        # ctx = [bs, ctx_len]
        # ques = [bs, ques_len]
        # char_ctx = [bs, ctx_len, ctx_word_len]
        # char_ques = [bs, ques_len, ques_word_len]
        
        ctx_len = ctx.shape[1]
        
        ques_len = ques.shape[1]
        
        ## GET WORD AND CHARACTER EMBEDDINGS
        
        ctx_word_embed = self.word_embedding(ctx)
        # ctx_word_embed = [bs, ctx_len, emb_dim]
        
        ques_word_embed = self.word_embedding(ques)
        # ques_word_embed = [bs, ques_len, emb_dim]        
        ctx_char_embed = self.character_embedding(char_ctx)
        # ctx_char_embed =  [bs, ctx_len, emb_dim]
        
        ques_char_embed = self.character_embedding(char_ques)
        # ques_char_embed = [bs, ques_len, emb_dim]
        
        ## CREATE CONTEXTUAL EMBEDDING
        
        ctx_contextual_inp = torch.cat([ctx_word_embed, ctx_char_embed],dim=2)
        # [bs, ctx_len, emb_dim*2]
        
        ques_contextual_inp = torch.cat([ques_word_embed, ques_char_embed],dim=2)
        # [bs, ques_len, emb_dim*2]
        
        ctx_contextual_emb = self.contextual_embedding(ctx_contextual_inp)
        # [bs, ctx_len, emb_dim*2]
        
        ques_contextual_emb = self.contextual_embedding(ques_contextual_inp)
        # [bs, ques_len, emb_dim*2]
        
        
        ## CREATE SIMILARITY MATRIX
        
        ctx_ = ctx_contextual_emb.unsqueeze(2).repeat(1,1,ques_len,1)
        # [bs, ctx_len, 1, emb_dim*2] => [bs, ctx_len, ques_len, emb_dim*2]
        
        ques_ = ques_contextual_emb.unsqueeze(1).repeat(1,ctx_len,1,1)
        # [bs, 1, ques_len, emb_dim*2] => [bs, ctx_len, ques_len, emb_dim*2]
        
        elementwise_prod = torch.mul(ctx_, ques_)
        # [bs, ctx_len, ques_len, emb_dim*2]
        
        alpha = torch.cat([ctx_, ques_, elementwise_prod], dim=3)
        # [bs, ctx_len, ques_len, emb_dim*6]
        
        similarity_matrix = self.similarity_weight(alpha).view(-1, ctx_len, ques_len)
        # [bs, ctx_len, ques_len]
        
        
        ## CALCULATE CONTEXT2QUERY ATTENTION
        
        a = F.softmax(similarity_matrix, dim=-1)
        # [bs, ctx_len, ques_len]
        
        c2q = torch.bmm(a, ques_contextual_emb)
        # [bs] ([ctx_len, ques_len] X [ques_len, emb_dim*2]) => [bs, ctx_len, emb_dim*2]
        
        
        ## CALCULATE QUERY2CONTEXT ATTENTION
        
        b = F.softmax(torch.max(similarity_matrix,2)[0], dim=-1)
        # [bs, ctx_len]
        
        b = b.unsqueeze(1)
        # [bs, 1, ctx_len]
        
        q2c = torch.bmm(b, ctx_contextual_emb)
        # [bs] ([bs, 1, ctx_len] X [bs, ctx_len, emb_dim*2]) => [bs, 1, emb_dim*2]
        
        q2c = q2c.repeat(1, ctx_len, 1)
        # [bs, ctx_len, emb_dim*2]
        
        ## QUERY AWARE REPRESENTATION
        
        G = torch.cat([ctx_contextual_emb, c2q, 
                       torch.mul(ctx_contextual_emb,c2q), 
                       torch.mul(ctx_contextual_emb, q2c)], dim=2)        # [bs, ctx_len, emb_dim*8]
        
        
        ## MODELING LAYER
        
        M, _ = self.modeling_lstm(G)
        # [bs, ctx_len, emb_dim*2]
        
        ## OUTPUT LAYER
        
        M2, _ = self.end_lstm(M)
        
        # START PREDICTION
        
        p1 = self.output_start(torch.cat([G,M], dim=2))
        # [bs, ctx_len, 1]
        
        p1 = p1.squeeze()
        # [bs, ctx_len]
        
        #p1 = F.softmax(p1, dim=-1)
        
        # END PREDICTION
        
        p2 = self.output_end(torch.cat([G, M2], dim=2)).squeeze()
        # [bs, ctx_len, 1] => [bs, ctx_len]
        
        #p2 = F.softmax(p2, dim=-1)
        
        
        return p1, p2
      

In [60]:
CHAR_VOCAB_DIM = len(char2idx)
EMB_DIM = 300
CHAR_EMB_DIM = 8
NUM_OUTPUT_CHANNELS = 300
KERNEL_SIZE = (8,5)
HIDDEN_DIM = 300
device = torch.device('cuda')

model = BiDAF(CHAR_VOCAB_DIM, 
              EMB_DIM, 
              CHAR_EMB_DIM, 
              NUM_OUTPUT_CHANNELS, 
              KERNEL_SIZE, 
              HIDDEN_DIM, 
              device).to(device)

In [ ]:
model

BiDAF(
  (word_embedding): Embedding(85812, 300)
  (character_embedding): CharacterEmbeddingLayer(
    (char_embedding): Embedding(193, 8, padding_idx=1)
    (char_convolution): Conv2d(1, 300, kernel_size=(8, 5), stride=(1, 1))
    (relu): ReLU()
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (contextual_embedding): ContextualEmbeddingLayer(
    (lstm): LSTM(600, 300, batch_first=True, bidirectional=True)
    (highway_net): HighwayNetwork(
      (flow_layer): ModuleList(
        (0): Linear(in_features=600, out_features=600, bias=True)
        (1): Linear(in_features=600, out_features=600, bias=True)
      )
      (gate_layer): ModuleList(
        (0): Linear(in_features=600, out_features=600, bias=True)
        (1): Linear(in_features=600, out_features=600, bias=True)
      )
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (similarity_weight): Linear(in_features=1800, out_features=1, bias=False)
  (modeling_lstm): LSTM(2400, 300, num_layers=2, batch_first=True, dropout=0.2,

In [61]:
import torch.optim as optim
from torch.autograd import Variable
optimizer = optim.Adadelta(model.parameters())

In [62]:
def train(model, train_dataset):
    print("Starting training ........")
   

    train_loss = 0.
    batch_count = 0
    model.train()
    for batch in train_dataset:
        
        optimizer.zero_grad()
    
        if batch_count % 500 == 0:
            print(f"Starting batch: {batch_count}")
        batch_count += 1
        
        context, question, char_ctx, char_ques, label, ctx_text, ans, ids = batch

        context, question, char_ctx, char_ques, label = context.to(device), question.to(device),\
                                   char_ctx.to(device), char_ques.to(device), label.to(device)


        preds = model(context, question, char_ctx, char_ques)

        start_pred, end_pred = preds

        s_idx, e_idx = label[:,0], label[:,1]

        loss = F.cross_entropy(start_pred, s_idx) + F.cross_entropy(end_pred, e_idx)


In [63]:
train(model, train_dataset)

Starting training ........
Starting batch: 0


RuntimeError: ignored